In [1]:
import fastai
from fastai.vision.all import * 

In [2]:
path = untar_data(URLs.ML_100k)

In [3]:
path

Path('/root/.fastai/data/ml-100k')

In [4]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])

In [5]:
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=['movie','title'], header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [8]:
from fastai.collab import *
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,693,Psycho (1960),5
1,880,"Scout, The (1994)",3
2,497,Judge Dredd (1995),4
3,864,I.Q. (1994),3
4,297,Liar Liar (1997),3
5,828,Margaret's Museum (1995),4
6,727,Volcano (1997),3
7,465,Glory (1989),3
8,749,"Rock, The (1996)",4
9,555,Anaconda (1997),4


In [9]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

#  Embedding Implementation from scratch

In [10]:
class T(Module):
    def __init__(self):
        self.a = torch.ones(3)

L(T().parameters())

(#0) []

In [11]:
class T(Module):
    def __init__(self):
        self.a = nn.Parameter(torch.ones(3))

t = T()
L(t.parameters())              #shows it in the list

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [12]:
class T(Module):
    def __init__(self):
        self.a = nn.Linear(2,3,bias = False)          # 2 input features and 3 output features

t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.5059, -0.5076],
        [-0.6240, -0.1684],
        [-0.2770,  0.5132]], requires_grad=True)]

In [13]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [14]:
def create_params(size):
    return nn.Parameter(torch.zeros(size).normal_(0,0.01))  # mean = 0 and standard deviation = 0.01

In [15]:
create_params([38,40])

Parameter containing:
tensor([[-0.0063, -0.0093, -0.0244,  ...,  0.0168, -0.0219, -0.0057],
        [-0.0182, -0.0025,  0.0092,  ...,  0.0011,  0.0108,  0.0061],
        [ 0.0065, -0.0032, -0.0054,  ..., -0.0059,  0.0142,  0.0039],
        ...,
        [-0.0016, -0.0077,  0.0062,  ...,  0.0101,  0.0086,  0.0043],
        [-0.0031, -0.0013,  0.0124,  ...,  0.0031,  0.0082, -0.0135],
        [ 0.0002,  0.0091, -0.0036,  ..., -0.0064, -0.0169, -0.0028]],
       requires_grad=True)

In [16]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]      # for indexing
        movies = self.movie_factors[x[:,1]]
        res = (users*movies).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [17]:
model = DotProductBias(n_users,n_movies, 50)
learn = Learner(dls, model, loss_func = MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd = 0.2)

epoch,train_loss,valid_loss,time
0,0.950164,0.930974,00:05
1,0.869464,0.881846,00:05
2,0.810497,0.840983,00:05
3,0.744324,0.812126,00:05
4,0.662283,0.807897,00:05


# Interpreting Embedding and Bias 

In [18]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
crap_movies = [dls.classes['title'][i] for i in idxs]

crap_movies

['Children of the Corn: The Gathering (1996)',
 'Barb Wire (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Leave It to Beaver (1997)',
 'Amityville II: The Possession (1982)']

In [19]:
idxs = movie_bias.argsort(descending=True)[:5]
best_movies = [dls.classes['title'][i] for i in idxs]
best_movies

['Good Will Hunting (1997)',
 'Shawshank Redemption, The (1994)',
 'Titanic (1997)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)"]

# Using Fastai Collab


In [20]:
learn = collab_learner(dls, n_factors = 50, y_range = (0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd = 0.2)

epoch,train_loss,valid_loss,time
0,0.943894,0.942554,00:05
1,0.904592,0.878006,00:05
2,0.814120,0.839092,00:05
3,0.733097,0.811821,00:05
4,0.670488,0.808739,00:05


In [21]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

# Embedding Distance


In [22]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Titanic (1997)'] #finding movie very similar to titanic through cosine similarity in embedding matrix
distances = nn.CosineSimilarity(dim = 1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending = True)[1]
dls.classes['title'][idx]

'Braveheart (1995)'

In [23]:
movie_factors[idx].shape

torch.Size([50])

In [24]:
movie_factors[idx][None].shape

torch.Size([1, 50])

In [25]:
movie_factors.shape

torch.Size([1665, 50])

In [26]:
x = torch.randn(1,5)
y = torch.randn(10,5)

In [27]:
x,y

(tensor([[-0.4678, -0.3603,  0.3683,  0.0036,  0.3654]]),
 tensor([[-0.1821, -0.7693,  1.1820, -2.1932, -0.5051],
         [-1.0809,  0.5989, -0.8404, -0.6813,  0.7957],
         [ 0.0693,  2.4848, -0.9003,  0.4538,  0.1962],
         [ 0.3867, -2.1705, -1.1292, -0.2307, -0.8067],
         [-1.3868, -0.5825,  0.6813, -0.4611, -0.1603],
         [ 1.3657,  1.0482,  2.2145,  0.2098, -1.0047],
         [ 0.7187, -1.7068,  0.3827,  1.0706,  1.3946],
         [ 1.5696,  1.4671, -0.9070,  0.1902,  0.5592],
         [ 1.3369,  1.6590,  2.9359, -0.8772,  0.3772],
         [ 1.9368,  0.9000, -0.9358, -0.6370, -0.4992]]))

In [28]:
z = nn.CosineSimilarity(dim = 1)(y, x)
z

tensor([ 0.2892,  0.1873, -0.5609, -0.0537,  0.7753, -0.2416,  0.4598, -0.7360,
        -0.0024, -0.9070])

In [29]:
z.shape

torch.Size([10])

# Using Deep Learning for the same problem

In [30]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act), 
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [31]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,1.017016,0.949028,00:07
1,0.904908,0.894950,00:06
2,0.859982,0.865990,00:07
3,0.832028,0.836262,00:06
4,0.775622,0.838827,00:06
